In [ ]:
from flask import Flask, request, jsonify
from model_manager import EthereumModelManager
from feature_server import EthereumFeatureServer
import logging
import socket
from datetime import datetime

# 配置日志
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = Flask(__name__)

# 初始化服务
try:
    model_manager = EthereumModelManager("models")
    feature_server = EthereumFeatureServer()
    logger.info("✅ 服务初始化成功")
except Exception as e:
    logger.error(f"❌ 服务初始化失败: {e}")
    exit(1)

@app.route('/health', methods=['GET'])
def health_check():
    """健康检查端点"""
    try:
        model_info = model_manager.get_model_info()
        
        return jsonify({
            'status': 'healthy',
            'timestamp': datetime.now().isoformat(),
            'model_version': model_info['version'],
            'model_type': model_info['model_type'],
            'feature_count': model_info['feature_count'],
            'service': 'CryptoGuard ML Prediction API'
        })
    except Exception as e:
        return jsonify({
            'status': 'unhealthy',
            'error': str(e),
            'timestamp': datetime.now().isoformat()
        }), 500

@app.route('/model/info', methods=['GET'])
def model_info():
    """获取模型信息"""
    try:
        info = model_manager.get_model_info()
        return jsonify(info)
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/predict', methods=['POST'])
def predict():
    """风险预测端点"""
    try:
        start_time = datetime.now()
        
        # 获取请求数据
        data = request.get_json()
        
        if not data:
            return jsonify({
                'success': False,
                'error': '请求体为空'
            }), 400
        
        address = data.get('address', '')
        if not address:
            return jsonify({
                'success': False,
                'error': '缺少 address 字段'
            }), 400
        
        # 特征工程
        features = feature_server.blockchain_to_model_features(address, data)
        
        # 模型预测
        prediction_result = model_manager.predict_risk(features)
        
        # 添加处理时间
        processing_time = (datetime.now() - start_time).total_seconds()
        prediction_result['processing_time_seconds'] = processing_time
        prediction_result['address'] = address
        
        logger.info(f"预测完成 - 地址: {address}, "
                   f"分数: {prediction_result['risk_score']:.1f}, "
                   f"耗时: {processing_time:.3f}s")
        
        return jsonify(prediction_result)
    
    except Exception as e:
        logger.error(f"预测请求处理失败: {e}")
        return jsonify({
            'success': False,
            'error': f'预测失败: {str(e)}',
            'timestamp': datetime.now().isoformat()
        }), 500

@app.route('/batch/predict', methods=['POST'])
def batch_predict():
    """批量预测端点"""
    try:
        data = request.get_json()
        
        if not data or 'addresses' not in data:
            return jsonify({'error': '缺少 addresses 字段'}), 400
        
        addresses = data['addresses']
        if not isinstance(addresses, list):
            return jsonify({'error': 'addresses 必须是列表'}), 400
        
        results = []
        for address in addresses[:10]:  # 限制批量处理数量
            features = feature_server.blockchain_to_model_features(address, {})
            prediction = model_manager.predict_risk(features)
            prediction['address'] = address
            results.append(prediction)
        
        return jsonify({
            'success': True,
            'results': results,
            'total_processed': len(results),
            'timestamp': datetime.now().isoformat()
        })
    
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/features/demo', methods=['GET'])
def features_demo():
    """特征演示端点 - 返回示例特征"""
    try:
        # 生成示例特征
        address = "0xd8dA6BF26964aF9D7eEd9e03E53415D37aA96045"
        features = feature_server.blockchain_to_model_features(address, {})
        
        return jsonify({
            'address': address,
            'features': features,
            'feature_count': len(features),
            'timestamp': datetime.now().isoformat()
        })
    except Exception as e:
        return jsonify({'error': str(e)}), 500

def safe_get_hostname():
    """安全获取主机名"""
    try:
        return socket.gethostname()
    except UnicodeDecodeError:
        return "localhost"

if __name__ == '__main__':
    print("🚀 启动 CryptoGuard ML API 服务...")
    print("📁 模型目录: model_serving/models")
    
    # 测试模型加载
    try:
        model_info = model_manager.get_model_info()
        print(f"✅ 模型加载成功: {model_info['model_type']} v{model_info['version']}")
        print(f"📊 特征数量: {model_info['feature_count']}")
    except Exception as e:
        print(f"❌ 模型加载失败: {e}")
        exit(1)
    
    # 安全启动
    host = '127.0.0.1'  # 使用IP地址避免主机名问题
    port = 5000
    
    print(f"🌐 API服务运行在: http://{host}:{port}")
    print("⏹️ 按 Ctrl+C 停止服务")
    
    try:
        app.run(host=host, port=port, debug=False)
    except Exception as e:
        print(f"❌ 服务器启动失败: {e}")
        print("🔄 尝试备用启动方式...")
        
        # 备用方案
        from werkzeug.serving import run_simple
        run_simple(host, port, app, use_reloader=False)